In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read data
df = pd.read_csv("data/gemstone.csv")

In [3]:
# drop id from dataframe
df.drop('id', axis = 1, inplace=True)
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z',
       'price'],
      dtype='object')

In [4]:
# Get unique data
df['color'].unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [ ]:
# Seprate data into X and Y
X = df.drop("price", axis=1)
y = df[['price']]

In [ ]:
# Get Categorical and numerical data
categorical_col = X.select_dtypes(include='object').columns
numerical_col = X.select_dtypes(exclude='object').columns

In [ ]:
# Define custom rank for each categorical data
cut = ['Fair','Good','Very Good','Premium','Ideal']
clarity = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
color = ['D','E','F','G','H','I','J']

In [ ]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import StandardScaler ## Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder ## Ordinal Encoding
## pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [32]:
# Create pipelines 

num_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut,color, clarity]))
    ]
)

In [33]:
preprocess = ColumnTransformer(
    [
        ('numerical_pipelines',num_pipeline,numerical_col),
        ('categorical_pipeline',cat_pipeline,categorical_col)
    ]
)

In [34]:
# Split Data into Train and Test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30)

In [35]:
# Transform data in one shot
preprocess.fit_transform(X_train)

array([[ 0.4737665 ,  3.22017586, -0.64163706, ...,  0.        ,
         2.        ,  3.        ],
       [-0.84569736,  0.07580627, -0.12039054, ...,  4.        ,
         3.        ,  2.        ],
       [ 0.73333317, -0.20163811,  0.92210251, ...,  2.        ,
         4.        ,  2.        ],
       ...,
       [ 0.45213595,  1.55550961, -0.64163706, ...,  1.        ,
         3.        ,  2.        ],
       [ 0.6684415 , -1.7738229 ,  1.44334904, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25746095,  0.81565794, -0.12039054, ...,  4.        ,
         3.        ,  2.        ]])

In [38]:
preprocess.transform(X_test)

array([[-0.56450014, -0.94148977, -0.64163706, ...,  3.        ,
         1.        ,  3.        ],
       [-0.17515015,  1.00062086, -0.12039054, ...,  2.        ,
         4.        ,  2.        ],
       [-1.06200291,  0.26076919, -0.12039054, ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [ 1.53366371,  0.35325065,  0.40085599, ...,  2.        ,
         3.        ,  4.        ],
       [ 1.62018593,  0.07580627,  0.92210251, ...,  3.        ,
         1.        ,  3.        ],
       [-0.97548069,  0.16828773,  1.44334904, ...,  2.        ,
         5.        ,  4.        ]])

In [41]:
preprocess.get_feature_names_out()

array(['numerical_pipelines__carat', 'numerical_pipelines__depth',
       'numerical_pipelines__table', 'numerical_pipelines__x',
       'numerical_pipelines__y', 'numerical_pipelines__z',
       'categorical_pipeline__cut', 'categorical_pipeline__color',
       'categorical_pipeline__clarity'], dtype=object)

In [43]:
# Store transformed data into new datasets
X_train = pd.DataFrame(preprocess.fit_transform(X_train), columns = preprocess.get_feature_names_out())
X_test = pd.DataFrame(preprocess.transform(X_test), columns = preprocess.get_feature_names_out())

In [45]:
# Model Training
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [47]:
# Evalueate the model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [48]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [49]:
trained_model_list = []
model_list = []
r2_list = []

In [54]:
# GEt the models dictionary values into list

print(list(models))
print(list(models.values()))

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']
[LinearRegression(), Lasso(), Ridge(), ElasticNet()]


In [55]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1020.9106955021983
MAE: 677.5652190317564
R2 score 93.63482324438802


Lasso
Model Training Performance
RMSE: 1020.8737429995507
MAE: 678.7748073387654
R2 score 93.63528401917503


Ridge
Model Training Performance
RMSE: 1020.910445208282
MAE: 677.5931366728804
R2 score 93.63482636545399


Elasticnet
Model Training Performance
RMSE: 1520.4410413453177
MAE: 1052.7631285487387
R2 score 85.88196907701382


